# CIVI Pytorch Image Classifier Workshop


## Contents
1. Setting Up Your Pytorch Environment.
2. Preparing Your Dataset.
3. Loading Pre-Trained Pytorch Models.
4. Loss Functions and Optimizers
5. Train/Test Your Models.

## Setting Up Your Pytorch Environment

To start, make sure you have the following libraries installed:
- Python (version 3.6 higher)
- [Pytorch (CUDA version)](https://pytorch.org)
    - Torchvision
    - Torchaudio
- [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit-archive)
    - [Check GPU Compatibility](https://developer.nvidia.com/cuda-gpus)

In [ ]:
# You should be able to run the following code if everything was installed correctly
import torch

print(f'GPU is available: {torch.cuda.is_available()}')
print(f'Number of GPUs: {torch.cuda.device_count()}')
print(f'Current GPU: {torch.cuda.current_device()}')

## Preparing Your Dataset

In this section, we will cover the basics of importing your **datasets** and preparing your **data loader**.

### Method 1: Using ``ImageFolder``

In [1]:
from torchvision import datasets
from torchvision import transforms

train_data = datasets.ImageFolder(root='dataset/train', 
                                  transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.
                                      transforms.Normalize((0.1307,), (0.3081,))
                                      ]))
test_data = datasets.ImageFolder(root='dataset/test',
                                 transform=transforms.Compose([
                                      transforms.ToTensor()
                                      ]))

### Method 2: Creating a Custom Dataset
A custom Dataset class must implement three functions: ``__init__``, ``__len__``, and ``__getitem__``. 

In [15]:
import os.path as osp
import torch
from torch.utils.data import Dataset
import cv2

class CustomMNIST(Dataset):

    def __init__(self,
                 annotations_path,
                 img_dir,
                 img_transform=None, 
                 target_transform=None):
        """Initializes the dataset

        Arguments:
            annotations_path {string} -- path to the annotations text file
            img_dir {string} -- path to the directory containing the images
            mode {string} -- current mode of the network
            new_size {int} -- rescaled size of the image
            image_transform {object} -- produces different dataset
            augmentation techniques
            target_transform {object} -- modifies the labels
        """

        self.img_dir = img_dir
        self.image_path = osp.join(img_dir, '%s')
        self.image_transform = img_transform
        self.target_transform = target_transform

        self.ids = []
        self.targets = []

        with open(annotations_path) as f:
            for line in f:
                values = line.split(' ')
                self.ids.append(values[0])
                self.targets.append(values[1].strip())

    def __len__(self):
        """Returns number of data in the dataset

        Returns:
            int -- number of data in the dataset
        """
        return len(self.ids)

    def __getitem__(self, index):
        """Returns an image and its corresponding class from the dataset

        Arguments:
            index {int} -- index of the item to be pulled from the list of ids

        Returns:
            torch.Tensor, string -- Tensor representation of the pulled image
            and string representation of the class of the image
        """
        
        image, target, _, _ = self.pull_item(index)

        return image, target


# HELPER FUNCTIONS
    def pull_item(self, index):
        """Returns an image, its corresponding class, height, and width from
        the dataset

        Arguments:
            index {int} -- index of the item to be pulled from the list of ids

        Returns:
            torch.Tensor, string, int, int -- Tensor representation of the
            pulled image, string representation of the class of the image,
            height of the image, width of the image
        """
        image_id = self.ids[index]

        image = cv2.imread(self.image_path % image_id)
        target = self.targets[index]
        height, width, _ = image.shape

        if self.image_transform is not None:
            image = self.image_transform(image)
            target = int(target)
            image = image[:, :, (2, 1, 0)]

        return image.permute(2, 0, 1), target, height, width

    def pull_image(self, index):
        """Returns an image from the dataset represented as an ndarray

        Arguments:
            index {int} -- index of the item to be pulled from the list of ids

        Returns:
            numpy.ndarray -- ndarray representation of the pulled image
        """

        image_id = self.ids[index]
        
        return cv2.imread(self.image_path % image_id, cv2.IMREAD_COLOR)

    def pull_target(self, index):
        """Returns a class corresponding to an image from the dataset

        Arguments:
            index {int} -- index of the item to be pulled from the list of ids

        Returns:
            string -- string representation of the class of an image from the
            dataset
        """
        return self.targets[index]

    def pull_tensor(self, index):
        """Returns an image from the dataset represented as a tensor

        Arguments:
            index {int} -- index of the item to be pulled from the list of ids

        Returns:
            torch.Tensor -- Tensor representation of the pulled image
        """
        return torch.Tensor(self.pull_image(index)).unsqueeze_(0)


In [16]:
from torchvision import transforms

train_data = CustomMNIST(annotations_path=osp.join('dataset', 'train_annotations.txt'),
                        img_dir=osp.join('dataset', 'train'),
                        img_transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307), (0.3081))
                                      ]))

test_data = CustomMNIST(annotations_path=osp.join('dataset', 'test_annotations.txt'),
                        img_dir=osp.join('dataset', 'test'),
                        img_transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307), (0.3081))
                                      ]))

### Method 3: Importing from Pytorch Directly
- [List of Available Datasets in Pytorch](https://pytorch.org/vision/stable/datasets.html)

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor


train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

### Creating the Data Loader

In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

## Loading Pre-Trained Pytorch Models
- [List of Available Models in Pytorch](https://pytorch.org/vision/stable/models.html)

In [ ]:
from torchvision import models

# Prints the list of available models
all_models = models.list_models(module=models)
print(all_models)

In [1]:
from torchvision import models
import torch.nn as nn

model = models.resnet50(weights='DEFAULT',
                        progress=True)

# Always change the final layer of your model to 
# match the number of classes in your dataset
model.fc = nn.Linear(in_features=2048,
                     out_features=10)

print(model) # Check the structure of the model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

### Creating Your Own Model

In [23]:
from torchvision import models
import torch.nn as nn

class My_Model(nn.Module):
    
    def __init__(self, num_of_classes):

        super(My_Model, self).__init__()
        
        # Using parts from pre-trained models
        features = list(models.resnet50(weights='DEFAULT',
                        progress=True).children())[:-2]
        
        self.backbone = nn.Sequential(*features)
        
        # Additional layers
        self.additiona_layers = nn.Sequential( 
            nn.Conv2d(2048, 1024, kernel_size=1, stride=1),
            nn.BatchNorm2d(1024),
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=1)
            )

        self.activation = nn.ReLU()
        self.fc = torch.nn.Linear(1024, num_of_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.additiona_layers(x)
        x = self.activation(x)
        x = self.fc(x.flatten(start_dim=1))
        
        return x

In [ ]:
my_model = My_Model(10) # Creating an instance of the model
print(my_model)

## Loss Functions and Optimizers
- [List of Available Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [List of Available Optimization Algorithms](https://pytorch.org/docs/stable/optim.html)

In [25]:
import torch.nn as nn
import torch.optim as optim

# Loss functions measure how close the predicted output
# is to the ground truth
criterion = nn.CrossEntropyLoss()

# Optimizers use the computed loss to update
# the parameters of the model
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Training/Testing Your Model

### Training Loop

In [ ]:
import torch
from tqdm import tqdm
import os

num_epochs = 1
model_save = 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# move model to gpu
model.to(device)
criterion.to(device)

# start training
for e in range(num_epochs):
    # set model in training mode
    model.train()

    running_loss = 0

    # Update learning rate
    if e + 1== 100:
        optimizer.param_groups[0]['lr'] /= 10
        
    for images, labels in tqdm(train_dataloader):

        # move data to gpu
        images = images.to(device)
        labels = torch.LongTensor(labels).to(device)

        # empty the gradients of the model through the optimizer
        optimizer.zero_grad()

        # forward pass
        output = model(images)

        # compute loss
        loss = criterion(output, labels.squeeze())

        # compute gradients using back propagation
        loss.backward()

        # update parameters
        optimizer.step()

        # update running loss
        running_loss += loss.item()

    # # save model weights 
    # if (e + 1) % model_save == 0:
    #     path = os.path.join(
    #         'weights',
    #         'resnet50/{}.pth'.format(e + 1)
    #     )

    #     torch.save(model.state_dict(), path)

    print(f'Epoch {e+1}/{num_epochs} | train loss: {running_loss / len(train_dataloader)}')

### Testing Loop

In [ ]:
import torch
from tqdm import tqdm
from sklearn.metrics import (accuracy_score, balanced_accuracy_score, f1_score)

# Move variables to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

y_true = torch.LongTensor([]).to(device)
y_pred = torch.LongTensor([]).to(device)

# load trained model
model.load_state_dict(torch.load('weights/resnet50/1.pth'))

# set model to evaluation mode
model.eval()

with torch.no_grad():
    for images, labels in tqdm(test_dataloader):
        images = images.to(device)
        labels = torch.LongTensor(labels).to(device)

        # call the model
        output = model(images)

        # retrieves predicted labels
        _, top_1_output = torch.max(output.data, dim=1)
        y_true = torch.cat((y_true, labels))
        y_pred = torch.cat((y_pred, top_1_output))

# move data back to cpu to compute for accuracy
y_true = y_true.cpu()
y_pred = y_pred.cpu()

# Performance Metrics
acc = accuracy_score(y_true, y_pred)
b_acc = balanced_accuracy_score(y_true, y_pred)
f1_mi = f1_score(y_true, y_pred, average='micro')
f1_ma = f1_score(y_true, y_pred, average='macro')
f1_w = f1_score(y_true, y_pred, average='weighted')

print(f'accuracy: {acc: .6f}, balanced accuracy: {b_acc: .6f}, f1_micro: {f1_mi: .6f}, f1_macro: {f1_ma: .6f}, f1_weighted: {f1_w: .6f}')

#### Using Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
print(f'Classification Report:\n{cr}')
disp.plot()
plt.title('Confusion Matrix')
plt.show()

### Additional Materials
- [Pytorch installation](https://www.youtube.com/watch?v=GMSjDTU8Zlc)
- [Datasets and Data Loader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
- [Using Pre-Trained Models](https://pytorch.org/vision/stable/models.html)
- [Building Custom Models](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html)
- [Training/Testing Loop](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)